In [1]:
!pip uninstall -y typing 
# this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'

!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

ERROR: Invalid requirement: '#'


  Running command git clone --filter=blob:none --quiet https://github.com/dreamquark-ai/tabnet.git 'C:\Users\Yeonjun Jeong\AppData\Local\Temp\pip-install-mgb58r0p\pytorch-tabnet_08c9b18bfa7745b4a678187b43e636a6'



  Cloning https://github.com/dreamquark-ai/tabnet.git (to revision develop) to c:\users\yeonjun jeong\appdata\local\temp\pip-install-mgb58r0p\pytorch-tabnet_08c9b18bfa7745b4a678187b43e636a6
  Resolved https://github.com/dreamquark-ai/tabnet.git to commit 40107a80b0be5ae865d945b85a52e5d99fc19a81
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pytorch_tabnet: filename=pytorch_tabnet-3.1.1-py3-none-any.whl size=42081 sha256=bab6dd5508a18878ce0663b72533570a63c3e980709cbfea02db9ee0b6c933eb
  Stored in directory: C:\Users\Yeonjun Jeong\AppData\Local\Temp\pip-ephem-wheel-cache-vhpbxtfk\wheels\33\ec\90\ae88ee166d5a2eabee63ae802d0a6cb752cf7d25a6a8613941
Successfully built pytorch_tabnet


In [85]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

import torch
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize, StandardScaler, MinMaxScaler,OrdinalEncoder
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import model_selection

import pandas as pd
import numpy as np
np.random.seed(0)

from tqdm.notebook import tqdm

import os

from matplotlib import pyplot as plt
%matplotlib inline

In [87]:
data_df = pd.read_csv('./pre_credit_df(fill_groupby)_final.csv', index_col=0)

In [88]:
categorical_columns = ['gender','car','reality','child_num','income_type','edu_type','family_type',
                        'house_type','work_phone','phone','email','occyp_type','dup','family_category','occyp_category']

In [89]:
data_df[categorical_columns] = data_df[categorical_columns].astype(object)

In [90]:
credit_Y = data_df['credit']
credit_X = data_df.drop(['credit'],axis=1)

In [91]:
encoder = StandardScaler()
numeric_features = ['income_total','DAYS_EMPLOYED','family_size','begin_month','Age','cards']
credit_X[numeric_features] = encoder.fit_transform(credit_X[numeric_features])

In [92]:
categorical_dims =  {}
for col in tqdm(credit_X.columns):
    if credit_X[col].dtype == object:
        l_enc = LabelEncoder()
        credit_X[col] = l_enc.fit_transform(credit_X[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)


  0%|          | 0/21 [00:00<?, ?it/s]

In [93]:
features = [ col for col in credit_X.columns] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [132]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(credit_X, credit_Y, test_size=0.2, random_state=0)
y_train = np.array(y_train).reshape((-1,1))
y_test = np.array(y_test).reshape((-1,1))
X_train = np.array(x_train)
X_test = np.array(x_test)

In [96]:
clf = TabNetMultiTaskClassifier(n_steps=1,
                                cat_idxs=cat_idxs,
                                cat_dims=cat_dims,
                                cat_emb_dim=1,
                                optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=2e-2),
                                scheduler_params={"step_size":50,
                                                  "gamma":0.9},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                mask_type='entmax', 
                                lambda_sparse=0, 
                       
                      )

C:\Users\Yeonjun Jeong\anaconda3\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [136]:
max_epochs = 1000
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set = [(X_test,y_test)],
    max_epochs=max_epochs ,
    patience=50, # please be patient ^^
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False
)

epoch 0  | loss: 0.92432 | val_0_logloss: 0.88085 |  0:00:05s
epoch 1  | loss: 0.88039 | val_0_logloss: 0.86777 |  0:00:10s
epoch 2  | loss: 0.86979 | val_0_logloss: 0.86298 |  0:00:15s
epoch 3  | loss: 0.85887 | val_0_logloss: 0.84979 |  0:00:20s
epoch 4  | loss: 0.84362 | val_0_logloss: 0.82801 |  0:00:26s
epoch 5  | loss: 0.82636 | val_0_logloss: 0.80842 |  0:00:31s
epoch 6  | loss: 0.81696 | val_0_logloss: 0.80212 |  0:00:37s
epoch 7  | loss: 0.8105  | val_0_logloss: 0.79635 |  0:00:43s
epoch 8  | loss: 0.81344 | val_0_logloss: 0.79921 |  0:00:49s
epoch 9  | loss: 0.80731 | val_0_logloss: 0.7951  |  0:00:55s
epoch 10 | loss: 0.80373 | val_0_logloss: 0.79853 |  0:01:01s
epoch 11 | loss: 0.80239 | val_0_logloss: 0.80102 |  0:01:07s
epoch 12 | loss: 0.80272 | val_0_logloss: 0.79162 |  0:01:12s
epoch 13 | loss: 0.80013 | val_0_logloss: 0.80341 |  0:01:18s
epoch 14 | loss: 0.79943 | val_0_logloss: 0.78786 |  0:01:24s
epoch 15 | loss: 0.7978  | val_0_logloss: 0.79791 |  0:01:30s
epoch 16

C:\Users\Yeonjun Jeong\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [138]:
!pip install optuna

     -------------------------------------- 308.2/308.2 KB 9.3 MB/s eta 0:00:00
     -------------------------------------- 210.7/210.7 KB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 81.0/81.0 KB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 KB ? eta 0:00:00
     -------------------------------------- 112.3/112.3 KB 6.8 MB/s eta 0:00:00
     -------------------------------------- 147.0/147.0 KB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 49.7/49.7 KB ? eta 0:00:00
     ---------------------------------------- 95.2/95.2 KB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11107 sha256=e29a2aae1f023eb47413c7f32792b3dc94201701ba26afe205b479501a809124
  Stored in directory: c:\users\yeonjun jeong\appdata\local\pip\cache\wheels\7f\1a\65\84ff8c386bec21fca6d220ea1f5498a036788

In [156]:
import optuna
from optuna import Trial, visualization
from sklearn.model_selection import StratifiedKFold

In [164]:
def Objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_steps = trial.suggest_int("n_steps", 1, 5, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    tabnet_params = dict(cat_idxs=cat_idxs,cat_dims=cat_dims,cat_emb_dim=1,
                     n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type=mask_type, n_shared=n_shared,
                     scheduler_params={"step_size":50,
                                                  "gamma":0.9},
                     scheduler_fn=torch.optim.lr_scheduler.StepLR,
                     verbose=0,
                     ) #early stopping
    random_state = 0
    num_folds= 5 # test_size = 0.2
    str_kf = StratifiedKFold(n_splits = num_folds, shuffle = True, random_state = random_state)
    CV_score_array    =[]
    for train_index, test_index in str_kf.split(credit_X, credit_Y):
        X_train, X_valid = credit_X.loc[train_index], credit_X.loc[test_index]
        y_train, y_valid = credit_Y.loc[train_index], credit_Y.loc[test_index]
        y_train = np.array(y_train).reshape((-1,1))
        y_valid = np.array(y_valid).reshape((-1,1))
        X_train = np.array(X_train)
        X_valid = np.array(X_valid)
        classifier = TabNetMultiTaskClassifier(**tabnet_params)
        classifier.fit(X_train=X_train, y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
                  )
        CV_score_array.append(classifier.best_cost)
    avg = np.mean(CV_score_array)
    return avg

In [166]:
study = optuna.create_study(direction="minimize", study_name='TabNet optimization')
study.optimize(Objective, n_trials=30)

[I 2022-05-10 00:46:57,480] A new study created in memory with name: TabNet optimization



Early stopping occurred at epoch 50 with best_epoch = 30 and best_val_0_logloss = 0.81075


C:\Users\Yeonjun Jeong\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 70 with best_epoch = 64 and best_val_0_logloss = 0.81006


C:\Users\Yeonjun Jeong\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...

C:\Users\Yeonjun Jeong\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 62 with best_epoch = 38 and best_val_0_logloss = 0.79966


C:\Users\Yeonjun Jeong\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 42 with best_epoch = 18 and best_val_0_logloss = 0.80484


C:\Users\Yeonjun Jeong\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2022-05-10 04:07:14,000] Trial 29 finished with value: 0.8016565084254038 and parameters: {'mask_type': 'entmax', 'n_steps': 4, 'gamma': 1.0, 'n_shared': 1, 'lambda_sparse': 5.708781880701445e-05, 'patience': 24, 'epochs': 87}. Best is trial 28 with value: 0.7945013596189942.


In [167]:
TabNet_params = study.best_params
print(TabNet_params)

{'mask_type': 'entmax', 'n_steps': 3, 'gamma': 1.0, 'n_shared': 2, 'lambda_sparse': 5.954980130207787e-05, 'patience': 24, 'epochs': 89}


In [168]:
final_params = dict( n_steps=TabNet_params['n_steps'], gamma=TabNet_params['gamma'],
                     lambda_sparse=TabNet_params['lambda_sparse'], optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type=TabNet_params['mask_type'], n_shared=TabNet_params['n_shared'],
                      scheduler_params={"step_size":50,
                                                  "gamma":0.9},
                     scheduler_fn=torch.optim.lr_scheduler.StepLR,
                     verbose=0,
                     )
epochs = TabNet_params['epochs']

In [169]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(credit_X, credit_Y, test_size=0.2, random_state=0)
y_train = np.array(y_train).reshape((-1,1))
y_test = np.array(y_test).reshape((-1,1))
X_train = np.array(x_train)
X_test = np.array(x_test)

In [170]:
clf = TabNetMultiTaskClassifier(**final_params)

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set = [(X_test,y_test)],
    patience=TabNet_params['patience'], max_epochs=epochs
)


Early stopping occurred at epoch 45 with best_epoch = 21 and best_val_0_logloss = 0.79013


C:\Users\Yeonjun Jeong\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
